# **Sección 1 : Interface web con GRADIO**

## Introducción a Gradio

Gradio ofrece dos API diferentes según el nivel de detalle que se busque:

- `gradio.Interface`: API de alto nivel que permite crear demos de ML simplemente proporcionando una lista de entradas y salidas.

- `gradio.Blocks`: API de bajo nivel que permite tener un control total sobre los flujos de datos y el diseño de la aplicación. Se pueden crear aplicaciones muy complejas de varios pasos utilizando Blocks (como si fueran bloques de construcción).

Comenzaremos utilizando `Interface` y al final mostraremos un ejemplo de `Blocks`.

## Instalamos y usamos Gradio


In [ ]:
!python --version

In [ ]:
!pip install gradio


Ejemplo usando una función para saludar que tiene `text` como input y `text` como output.


In [ ]:
import gradio as gr

def saludo(nombre):
    return "Hola " + nombre + ", ¿Como estas??? "


demo = gr.Interface(
    fn=saludo,
    inputs = "text",
    outputs = "text"
)

demo.launch()

La clase de `gr.Interface` es una forma fácil de crear demos que pueden ser desde una calculadora hasta una aplicación para reconocimiento de voz.

Se inicializa con tres parámetros necesarios:


*   `fn`: la función.

*   `inputs`: qué componente(s) usar para los inputs de la función, por ejemplo, "texto", "imagen" o "audio"
* `outputs`: qué componente(s) usar para los outputs de la función, por ejemplo, "texto", "imagen" o "etiqueta"


Gradio incluye más de 20 componentes diferentes, la mayoría de los cuales se pueden utilizar como inputs y outputs. En la documentación está la [lista completa](https://gradio.app/docs/#components).

Ejemplo 2:

In [ ]:
import gradio as gr

def saludo(nombre):
    return "Hola " + nombre + ", ¿Como estas??? "


demo = gr.Interface(
    fn=saludo,
    inputs = gr.components.Textbox(lines=10, placeholder="Coloca tu nombre aqui"),
    outputs = "text",
    title="Aplicacion de prueba",
    description= "ingrese su nombre para ser saludado"
)

demo.launch()

# **Sección 2: usando modelos preentrenados HuhuggingFace**

Veremos como utilizar la biblioteca `transformers` para cargar un clasificador de texto pre-entrenado y luego realizar una predicción de clasificación de texto utilizando ese clasificador. En este caso, estáras utilizando el método `pipeline` para cargar un clasificador de texto pre-entrenado sin necesidad de configurar un modelo por separado. Aquí tienes una explicación paso a paso de lo que hace el código:
https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment






Este es un modelo bert-base-multilingüe-sin mayúsculas optimizado para el análisis de sentimientos sobre reseñas de productos en seis idiomas: inglés, holandés, alemán, francés, español e italiano. Predice el sentimiento de la reseña como una cantidad de estrellas (entre 1 y 5).

Este modelo está pensado para su uso directo como modelo de análisis de opiniones para reseñas de productos en cualquiera de los seis idiomas anteriores o para un mayor ajuste de tareas de análisis de opiniones relacionadas.




1. **Instalar la biblioteca `transformers`:**

```python
!pip install transformers # nos puede faltar por ejemplo instalar aparte torch

!pip install git+https://github.com/huggingface/transformers.git #diractamente del repositorio
```


- Utilizamos `!pip install` para instalar la biblioteca `transformers`.

In [ ]:
!pip install transformers

2. **Importar la biblioteca `pipeline` y cargar el clasificador:**

```python
from transformers import pipeline # permite encadenar tareas y bajar modelos

clasificador = pipeline([tarea],[modelo]) #Espera recibir la tupla compuesta por tipo de tarea y modelo a utilizar
```


In [ ]:
from transformers import pipeline

clasificador = pipeline("text-classification", #tarea
                        model="nlptown/bert-base-multilingual-uncased-sentiment") #modelo


3. **Realizar una predicción de clasificación:**

```python
resultado = clasificador("Estoy muy contento de aprender sobre la biblioteca transformers")
```

In [ ]:
secuencia = "Estoy muy contento de aprender sobre la biblioteca transformers"

resultado = clasificador(secuencia)



4. **Mostrar el resultado:**


In [ ]:
print(resultado)

## Una demo usando Gradio Blocks

Creamos un demo que recibe dos modelos. Puede transcribir una voz y también puede clasificar el sentimiento de un texto.

Usaremos el modelo https://huggingface.co/facebook/wav2vec2-large-xlsr-53-spanish para hacer transcripcion de audio a texto

y el modelo https://huggingface.co/pysentimiento/robertuito-sentiment-analysis para clasificar sentimiento entre positivo, neutro o negativo. Este modelo a sido desarrllado por gente de la UBA

Primero instalamos las dependencias del proyecto

In [ ]:
# con %%capture evitamos que se vea por pantalla la instalación
%%capture
!pip install gradio
!pip install transformers
!pip install torch

Importamos y generamos las instancias de los modelos.
- 'trans' sera para transcribir audio a texto
- clasificador sera para clasificar sentimiento

In [ ]:
import gradio as gr
from transformers import pipeline

trans = pipeline("automatic-speech-recognition", model = "facebook/wav2vec2-large-xlsr-53-spanish")
clasificador = pipeline("text-classification", model = "pysentimiento/robertuito-sentiment-analysis")

Ahora generemos las funciones necearias para cada caso que luego utilizaremos en la interface de GRADIO

In [ ]:
def audio_a_texto(audio):
  text = trans(audio)["text"]
  return text

def texto_a_sentimiento(text):
  return clasificador(text)[0]["label"]

Generemos el Bloque que contiene nuestro proyecto y ejecutamos con demo.launc()

In [ ]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Demo de uso de Bloques e integracion de Modelos")
  audio = gr.Audio(sources="microphone", type="filepath")

  texto = gr.Textbox()
  b1 = gr.Button("Transcribir")
  b1.click(audio_a_texto, inputs=audio, outputs=texto)

  label = gr.Label()
  b2 = gr.Button("Clasificar")
  b2.click(texto_a_sentimiento, inputs=texto, outputs=label)

demo.launch()

Hagamos un aplicación de blocks un poco más interesante. organizando los componentes en Tabs

In [ ]:
demo = gr.Blocks()

with demo:
  gr.Markdown("Este es el segundo demo con Blocks")
  with gr.Tabs():
    with gr.TabItem("Transcribe audio en español"):
      with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")
        transcripcion = gr.Textbox()
      b1 = gr.Button("Transcribir")

    with gr.TabItem("Análisis de sentimiento en español"):
      with gr.Row():
        texto = gr.Textbox()
        label = gr.Label()
      b2 = gr.Button("Sentimiento")

    # Asignar funciones a los botones
    b1.click(audio_a_texto, inputs = audio, outputs=transcripcion)
    b2.click(texto_a_sentimiento, inputs=texto, outputs=label)

demo.launch()

    # Desafio:
    # Crear un enlace entre la salida de la primera pestaña y la entrada de la segunda pestaña
    # Falta desarrollar
